# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd

df = pd.read_csv('homepage_actions.csv')
df.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# Check for unique viewers who clicked
unique_ids = df['id'].nunique()
print(f"Total unique users: {unique_ids}")

# Check for any users in both control and experiment groups
overlap = df.groupby('id')['group'].nunique().gt(1).sum()
print(f"Users appearing in both groups: {overlap}")


Total unique users: 6328
Users appearing in both groups: 0


In [7]:
# Total number of viewers per group
group_counts = df.groupby('group')['id'].nunique()
print(group_counts)

# Number of clicks in each group
clicks_per_group = df[df['action'] == 'click'].groupby('group')['id'].nunique()
print(clicks_per_group)

group
control       3332
experiment    2996
Name: id, dtype: int64
group
control       932
experiment    928
Name: id, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [8]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Calculate click counts and total counts per group
click_counts = df[df['action'] == 'click'].groupby('group')['id'].nunique()
total_counts = df.groupby('group')['id'].nunique()

# Perform z-test
z_stat, p_value = proportions_ztest(count=click_counts, nobs=total_counts)
print(f"Z-statistic: {z_stat}, P-value: {p_value}")


Z-statistic: -2.618563885349469, P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [9]:
#Your code here
control_ctr = click_counts['control'] / total_counts['control']
expected_clicks_experiment = control_ctr * total_counts['experiment']
print(f"Expected clicks for experiment group: {expected_clicks_experiment}")


Expected clicks for experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
import numpy as np

std_dev = np.sqrt(total_counts['experiment'] * control_ctr * (1 - control_ctr))
print(f"Standard deviation of expected clicks: {std_dev}")


Standard deviation of expected clicks: 24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
actual_clicks_experiment = click_counts['experiment']
z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev

# Calculate p-value based on z-score
from scipy.stats import norm
p_value_verification = 2 * norm.sf(abs(z_score))
print(f"Verification Z-score: {z_score}, Verification P-value: {p_value_verification}")


Verification Z-score: 3.6625360854823588, Verification P-value: 0.00024973056013902396


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The analysis confirmed that the experimental homepage design led to a significantly higher click-through rate compared to the control. This result was robust, as evidenced by both the z-test and the verification steps using expected values and standard deviation.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.